<a href="https://colab.research.google.com/github/TejaswiniMalgu/Development-of-Interactive-Cyber-Threat-Visualization-Dashboard/blob/main/SQL_TASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from google.colab import files
import io

print("Upload the cyber threat CSV file")
uploaded = files.upload()

for file_name in uploaded:
    df = pd.read_csv(io.StringIO(uploaded[file_name].decode("utf-8")))

print("Dataset loaded successfully")
df.head()



Upload the cyber threat CSV file


In [3]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

np.random.seed(42)

records = 300

attack_types = ['DDoS','Malware','Phishing','SQL Injection','XSS','Brute Force','Ransomware','Privilege Escalation']
severity_levels = ['Low','Medium','High','Critical']
protocols = ['TCP','UDP','HTTP','HTTPS','SMTP','SSH','FTP']
countries = ['India','USA','Germany','Australia','Singapore','UK','Canada']
statuses = ['Blocked','Detected','Mitigated','Successful']

def random_ip():
    return ".".join(map(str, np.random.randint(1,255,4)))

data = []

for i in range(records):
    data.append({
        'incident_id': f'INC-{10000+i}',
        'timestamp': datetime.now() - timedelta(days=random.randint(1,180)),
        'source_ip': random_ip(),
        'destination_ip': random_ip(),
        'attack_type': random.choice(attack_types),
        'severity': random.choice(severity_levels),
        'protocol': random.choice(protocols),
        'destination_port': random.randint(100, 9999),
        'source_country': random.choice(countries),
        'packet_count': random.randint(100, 80000),
        'data_volume_mb': round(random.uniform(0.5, 800), 2),
        'avg_packet_size_bytes': random.randint(64, 1500),
        'response_time_ms': random.randint(1, 7000),
        'attack_status': random.choice(statuses),
        'is_malicious': random.choice([0,1])
    })

df = pd.DataFrame(data)
df.head()


,incident_id,timestamp,source_ip,destination_ip,attack_type,severity,protocol,destination_port,source_country,packet_count,data_volume_mb,avg_packet_size_bytes,response_time_ms,attack_status,is_malicious
0,INC-10000,2025-12-05 15:08:00.942595,103.180.93.15,107.72.189.21,Phishing,Low,TCP,6010,Singapore,56743,293.45,390,3140,Detected,1
1,INC-10001,2025-12-19 15:08:00.942941,103.122.211.215,75.203.88.117,DDoS,Medium,FTP,410,Canada,55613,197.71,941,6064,Mitigated,0
2,INC-10002,2025-07-26 15:08:00.943180,100.104.152.131,150.53.2.88,Brute Force,High,HTTPS,4349,UK,39154,275.63,1262,5260,Blocked,0
3,INC-10003,2025-07-13 15:08:00.943636,236.158.38.130,192.188.21.161,DDoS,Medium,SSH,1310,Singapore,52398,563.84,188,2881,Successful,0
4,INC-10004,2025-12-27 15:08:00.943768,204.58.22.253,236.89.49.219,XSS,Critical,HTTP,543,USA,72206,714.37,1167,1492,Detected,1


In [4]:
# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

# One-hot encode categorical columns
categorical_cols = [
    'incident_id','source_ip','destination_ip',
    'attack_type','severity','protocol',
    'source_country','attack_status'
]

df_encoded = pd.get_dummies(df, columns=categorical_cols)

print("Encoded dataset shape:", df_encoded.shape)


Encoded dataset shape: (300, 937)


In [5]:
import sqlite3

conn = sqlite3.connect(':memory:')
df_encoded.to_sql('incidents', conn, if_exists='replace', index=False)

print("SQLite database and table created successfully")


SQLite database and table created successfully


In [6]:
pd.read_sql("SELECT COUNT(*) AS total_incidents FROM incidents;", conn)


,total_incidents
0,300


In [7]:
pd.read_sql("""
SELECT is_malicious, COUNT(*) AS count
FROM incidents
GROUP BY is_malicious;
""", conn)


,is_malicious,count
0,0,137
1,1,163


In [8]:
pd.read_sql("""
SELECT
CASE
 WHEN severity_Low=1 THEN 'Low'
 WHEN severity_Medium=1 THEN 'Medium'
 WHEN severity_High=1 THEN 'High'
 WHEN severity_Critical=1 THEN 'Critical'
END AS severity,
COUNT(*) AS count
FROM incidents
GROUP BY severity;
""", conn)


,severity,count
0,Critical,75
1,High,66
2,Low,82
3,Medium,77


In [9]:
pd.read_sql("""
SELECT
CASE
 WHEN "attack_type_DDoS"=1 THEN 'DDoS'
 WHEN "attack_type_Malware"=1 THEN 'Malware'
 WHEN "attack_type_Phishing"=1 THEN 'Phishing'
 WHEN "attack_type_SQL Injection"=1 THEN 'SQL Injection'
 WHEN "attack_type_XSS"=1 THEN 'XSS'
 WHEN "attack_type_Brute Force"=1 THEN 'Brute Force'
 ELSE 'Other'
END AS attack_type,
COUNT(*) AS count
FROM incidents
GROUP BY attack_type;
""", conn)


,attack_type,count
0,Brute Force,37
1,DDoS,47
2,Malware,36
3,Other,56
4,Phishing,35
5,SQL Injection,38
6,XSS,51


In [10]:
pd.read_sql("""
SELECT
CASE
 WHEN source_country_India=1 THEN 'India'
 WHEN source_country_USA=1 THEN 'USA'
 WHEN source_country_Germany=1 THEN 'Germany'
 WHEN source_country_Australia=1 THEN 'Australia'
 WHEN source_country_Singapore=1 THEN 'Singapore'
 ELSE 'Other'
END AS country,
COUNT(*) AS count
FROM incidents
GROUP BY country
ORDER BY count DESC
LIMIT 5;
""", conn)


,country,count
0,Other,85
1,India,51
2,Germany,48
3,Singapore,40
4,USA,39


In [11]:
pd.read_sql("""
SELECT
CASE
 WHEN attack_status_Blocked=1 THEN 'Blocked'
 WHEN attack_status_Mitigated=1 THEN 'Mitigated'
 WHEN attack_status_Detected=1 THEN 'Detected'
 WHEN attack_status_Successful=1 THEN 'Successful'
END AS status,
COUNT(*) AS count
FROM incidents
WHERE is_malicious=1
GROUP BY status;
""", conn)


,status,count
0,Blocked,37
1,Detected,45
2,Mitigated,37
3,Successful,44


In [12]:
pd.read_sql("""
SELECT destination_port, packet_count
FROM incidents
WHERE destination_port>1024 AND packet_count>10000
ORDER BY packet_count DESC
LIMIT 10;
""", conn)


,destination_port,packet_count
0,2926,79287
1,1595,79155
2,7372,78432
3,3041,78287
4,2716,78004
5,9242,77931
6,5387,77656
7,7097,77433
8,7194,77355
9,7946,77222


In [13]:
pd.read_sql("""
SELECT response_time_ms, data_volume_mb
FROM incidents
WHERE response_time_ms >
 (SELECT AVG(response_time_ms) FROM incidents)
ORDER BY response_time_ms DESC
LIMIT 10;
""", conn)


,response_time_ms,data_volume_mb
0,6999,669.70
1,6984,765.74
2,6871,567.16
3,6857,442.31
4,6848,482.32
5,6839,157.67
6,6833,106.83
7,6796,586.53
8,6739,256.23
9,6738,74.48


In [14]:
pd.read_sql("""
SELECT response_time_ms, data_volume_mb
FROM incidents
WHERE response_time_ms>5000 AND data_volume_mb<50
ORDER BY response_time_ms DESC;
""", conn)


,response_time_ms,data_volume_mb
0,6365,7.34
1,5961,38.52
2,5795,8.79
3,5331,44.42
4,5253,7.69
5,5226,29.70
6,5133,20.71
7,5065,11.22
